## Neurips AIDO-3 DeepRacer Racing Challenge Submission

This notebook walks you through the submission process, for a valid submission we need the following
1. Model checkpoint
2. Action space metadata json
3. Evaluation logs from the simulation


### Step 1:  set parameters
Copy the AWS RoboMaker evaluation simulation identification number. AWS SageMaker training job saves checkpoint and frozen graphs into an S3 model bucket. Copy the bucket and prefix from your training job.

In [1]:
# FILL these in
stream_name = '<Evaluation Job SIMULATION ID>'
s3_bucket = '<FILL IN YOUR S3 Bucket>'
s3_prefix = '<FILL IN YOUR S3 Prefix>'

### Step 2:  Download model and logs

We have libraries that make it easy for you to download the logs. To ensure it works smoothly, ensure the following
1. Your boto library version is >= '1.9.133'
2. Ensure the boto credentials you are using can download cloudwatch logs
3. If you are using a cloud notebook instance (either ec2 or SageMaker), ensure that your IAM role has access to S3 bucket that you configured above, and cloudwatch logs

Your IAM role should be similar to the following 

```
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents",
                "logs:DescribeLogStreams",
                "logs:FilterLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:*:*:log-group:/aws/robomaker/*:log-stream:*"
            ]
        }
```
or have access to all the cloudwatch logs 

```
 "Resource": [
                "arn:aws:logs:*:*:*"
            ]
```

### Imports

In [20]:
from datetime import datetime
%matplotlib inline

import os
os.getcwd()

'/home/ec2-user/SageMaker/challenge_submit'

In [21]:
import cw_utils

In [22]:
# Make sure your boto version is >= '1.9.133'
cw_utils.boto3.__version__

'1.10.1'

### Create a submission folder

In [7]:
submit_folder_name = 'submit/'
if os.path.exists(submit_folder_name):
    !rm -r submit
!mkdir submit


Your model files are in the following s3 bucket. Ensure that the model file you choose to submit is in the s3 bucket.

In [8]:
print('Your model s3 bucket: s3://%s/%s/model' %(s3_bucket, s3_prefix))

Your model s3 bucket: s3://sahika-lidar-oldcar-tf12-wide/sahika-lidar-oldcar-tf12-wide-sagemaker-191107-051402/model


Enter the name of the model file and prefix for checkpoint files.

In [9]:
model_name_in_s3_frozen_pb = "model_0.pb"
model_name_in_s3_checkpoint_prefix = "0_Step-0.ckpt"

model_name_in_s3_checkpoint_prefix += '*'
!aws s3 sync s3://{s3_bucket}/{s3_prefix}/model $submit_folder_name --exclude "*" --include $model_name_in_s3_frozen_pb --include "*.json" --include $model_name_in_s3_checkpoint_prefix


download: s3://sahika-lidar-oldcar-tf12-wide/sahika-lidar-oldcar-tf12-wide-sagemaker-191107-051402/model/0_Step-0.ckpt.index to submit/0_Step-0.ckpt.index
download: s3://sahika-lidar-oldcar-tf12-wide/sahika-lidar-oldcar-tf12-wide-sagemaker-191107-051402/model/0_Step-0.ckpt.meta to submit/0_Step-0.ckpt.meta
download: s3://sahika-lidar-oldcar-tf12-wide/sahika-lidar-oldcar-tf12-wide-sagemaker-191107-051402/model/model_0.pb to submit/model_0.pb
download: s3://sahika-lidar-oldcar-tf12-wide/sahika-lidar-oldcar-tf12-wide-sagemaker-191107-051402/model/model_metadata.json to submit/model_metadata.json
download: s3://sahika-lidar-oldcar-tf12-wide/sahika-lidar-oldcar-tf12-wide-sagemaker-191107-051402/model/0_Step-0.ckpt.data-00000-of-00001 to submit/0_Step-0.ckpt.data-00000-of-00001


Copy the log file to the submission folder.

In [10]:
fname = submit_folder_name + '/deepracer-%s.log' %stream_name
cw_utils.download_log(fname, stream_prefix=stream_name)
 

Writing to ...  submit//deepracer-sim-bc74dbfkwh5q.log


### Custom model submission

if you didn't modify the algorithm from (ClippedPPO), then just run next cell without any modification. Else please specify the output head of the algorithm used.

In [ ]:
## To run the neural network on the robocar, we need to name of the output layer.

output_head = "main_level/agent/main/online/network_1/ppo_head_0/policy"

#uncomment for DQN
#output_head = "main_level/agent/main/online/network_0/q_values_head_0/q_values_head_0_target"

output_head_file = submit_folder_name + 'output_head.txt'

!echo $output_head > $output_head_file

In [12]:
!tar -czvf submit.tar.gz $submit_folder_name

submit/
submit/0_Step-0.ckpt.index
submit/deepracer-sim-bc74dbfkwh5q.log
submit/model_metadata.json
submit/0_Step-0.ckpt.data-00000-of-00001
submit/0_Step-0.ckpt.meta
submit/model_0.pb


### Upload and Submit

Go to [LINK] and upload the tar.gz file by filling out the form